## Cleaning Data from Provience Weather Stations

## Connect and set up SQL 

In [340]:
import sqlite3
con = sqlite3.connect("WEATHER.db") #create new database called Weather or connect to it if it already exists

cursor_obj = con.cursor() #create a cursor to fetch results from SQL queries

In [341]:
!pip install "sqlalchemy<2"

#allows use of sql magic command
%load_ext sql
#connects to database
%sql sqlite:///WEATHER.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @WEATHER.db'

## Load data using Pandas

In [342]:
import csv, pandas as pd, numpy as np

df1 = pd.read_csv("/Users/maggiesullens/Library/Mobile Documents/com~apple~CloudDocs/Spruce Budworm/Retrieved weather/qc_weather.csv")
df1.rename(columns={"Unnamed: 0": "ID"}, inplace=True)
df1.to_sql("WEATHER", con, if_exists='replace', index=False)


7805150

## Show glipse of data

In [343]:
%%sql 
SELECT *
FROM WEATHER LIMIT 3;


 * sqlite:///WEATHER.db
Done.


ID,prov,station_name,station_id,lat,lon,elev,date,max_temp,mean_temp,min_temp,total_precip
1,QC,DALHOUSIE STATION,5199,45.3,-74.47,70.0,1974-09-01,21.1,13.4,5.6,0.0
2,QC,DALHOUSIE STATION,5199,45.3,-74.47,70.0,1974-09-02,16.1,13.1,10.0,8.6
3,QC,DALHOUSIE STATION,5199,45.3,-74.47,70.0,1974-09-03,15.6,12.5,9.4,3.3


## Make Daily Averages Dataset for Max Temp, Min Temp, Mean Temp, and Total Precip

In [344]:
%%sql
DROP TABLE IF EXISTS DAY_AVERAGE_WEATHER;

CREATE TABLE DAY_AVERAGE_WEATHER AS
SELECT 
  date,
  COUNT(*) AS group_size,
  ROUND(AVG(max_temp), 4) AS avg_max_temp,
  ROUND(AVG(min_temp), 4) AS avg_min_temp,
  ROUND(AVG(mean_temp), 4) AS avg_mean_temp,
  ROUND(AVG(total_precip), 4) AS avg_total_precip
FROM WEATHER
GROUP BY date;
SELECT * FROM DAY_AVERAGE_WEATHER ORDER BY date LIMIT 100;


 * sqlite:///WEATHER.db
Done.
Done.
Done.


date,group_size,avg_max_temp,avg_min_temp,avg_mean_temp,avg_total_precip
1871-07-01,1,28.3,13.3,20.8,0.0
1871-07-02,1,26.1,19.4,22.8,0.0
1871-07-03,1,28.9,11.7,20.3,0.0
1871-07-04,1,26.7,17.2,22.0,0.0
1871-07-05,1,27.8,16.7,22.3,19.6
1871-07-06,1,28.3,16.7,22.5,0.0
1871-07-07,1,23.3,20.6,22.0,19.1
1871-07-08,1,30.0,16.7,23.4,0.0
1871-07-09,1,26.7,21.1,23.9,1.0
1871-07-10,1,24.4,15.0,19.7,0.0


## Check to see if there are any duplicate days

In [345]:
%%sql
SELECT date, COUNT(*) AS occurrences
FROM DAY_AVERAGE_WEATHER
GROUP BY date
HAVING COUNT(*) > 1;

 * sqlite:///WEATHER.db
Done.


date,occurrences


## Make Monthly Information

In [346]:
%%sql
DROP TABLE IF EXISTS MONTHLY_WEATHER_VARS;

CREATE TABLE MONTHLY_WEATHER_VARS AS
SELECT 
  strftime('%Y', date) AS year,
  strftime('%m', date) AS month,
  COUNT(*) AS group_size,
  ROUND(AVG(avg_max_temp), 2) AS month_avg_max_temp,
  ROUND(AVG(avg_min_temp), 2) AS month_avg_min_temp,
  ROUND(AVG(avg_mean_temp), 2) AS month_avg_mean_temp,
  MAX(avg_max_temp) AS month_max_temp,
  MIN(avg_min_temp) AS month_min_temp,
  ROUND(AVG(avg_max_temp - avg_min_temp), 2) AS mean_diurnal_range,
  ROUND(SUM(avg_total_precip), 2) AS month_total_precip
FROM DAY_AVERAGE_WEATHER
GROUP BY year, month;

SELECT * FROM MONTHLY_WEATHER_VARS ORDER BY year, month LIMIT 25;

 * sqlite:///WEATHER.db
Done.
Done.
Done.


year,month,group_size,month_avg_max_temp,month_avg_min_temp,month_avg_mean_temp,month_max_temp,month_min_temp,mean_diurnal_range,month_total_precip
1871,07,31,26.19,16.92,21.57,35.0,11.7,9.27,126.3
1871,08,31,26.51,17.02,21.79,32.2,11.7,9.48,95.5
1871,09,30,21.28,10.68,16.0,32.8,3.3,10.6,33.0
1871,10,31,14.08,6.4,10.26,28.3,-1.1,7.68,85.2
1871,11,30,3.52,-3.78,-0.12,11.1,-21.7,7.3,74.3
1871,12,31,-4.42,-11.04,-7.75,7.8,-30.6,6.61,75.2
1872,01,31,-3.11,-11.43,-7.28,5.6,-23.3,8.32,49.2
1872,02,29,-0.37,-12.1,-6.26,7.2,-18.9,11.74,83.3
1872,03,31,-0.47,-11.55,-6.02,11.1,-28.9,11.08,76.9
1872,04,30,13.32,1.11,7.23,26.7,-3.9,12.2,36.1


## Make Quarterly Variable Dataset

In [347]:
%%sql
DROP TABLE IF EXISTS QUARTERLY;

CREATE TABLE QUARTERLY AS
SELECT 
  strftime('%Y', date) AS year,
  CASE 
    WHEN CAST(strftime('%m', date) AS INTEGER) IN (1,2,3) THEN 1
    WHEN CAST(strftime('%m', date) AS INTEGER) IN (4,5,6) THEN 2
    WHEN CAST(strftime('%m', date) AS INTEGER) IN (7,8,9) THEN 3
    WHEN CAST(strftime('%m', date) AS INTEGER) IN (10,11,12) THEN 4
END AS quarter,
COUNT(*) AS group_size,
  ROUND(AVG(avg_max_temp), 2) AS quarter_avg_max_temp,
  ROUND(AVG(avg_min_temp), 2) AS quarter_avg_min_temp,
  ROUND(AVG(avg_mean_temp), 2) AS quarter_avg_mean_temp,
  ROUND(SUM(avg_total_precip), 2) AS quarter_total_precip
FROM DAY_AVERAGE_WEATHER 
GROUP BY year, quarter;

SELECT * FROM QUARTERLY ORDER BY year, quarter LIMIT 10;

 * sqlite:///WEATHER.db
Done.
Done.
Done.


year,quarter,group_size,quarter_avg_max_temp,quarter_avg_min_temp,quarter_avg_mean_temp,quarter_total_precip
1871,3,92,24.69,14.92,19.83,254.8
1871,4,92,4.4,-2.79,0.81,234.7
1872,1,91,-1.33,-11.68,-6.53,209.4
1872,2,91,19.57,8.76,14.19,140.4
1872,3,92,25.78,16.29,21.06,332.7
1872,4,92,3.7,-3.69,0.01,367.0
1873,1,90,-2.36,-10.65,-6.52,307.1
1873,2,91,17.51,7.53,12.54,188.7
1873,3,92,23.99,13.77,18.91,220.95
1873,4,92,4.02,-4.14,-0.06,329.7


## Make Yearly Variable Dataset

In [348]:
%%sql
DROP TABLE IF EXISTS YEARLY_WEATHER;

CREATE TABLE YEARLY_WEATHER AS
SELECT 
  strftime('%Y', date) AS year,
  COUNT(*) AS group_size,
  ROUND(AVG(avg_max_temp), 2) AS year_avg_max_temp,
  ROUND(AVG(avg_min_temp), 2) AS year_avg_min_temp,
  ROUND(AVG(avg_mean_temp), 2) AS year_avg_mean_temp,
  ROUND(SUM(avg_total_precip), 2) AS year_total_precip
FROM DAY_AVERAGE_WEATHER 
GROUP BY year;

ALTER TABLE YEARLY_WEATHER ADD COLUMN max_temp_of_warmest_month REAL;
ALTER TABLE YEARLY_WEATHER ADD COLUMN min_temp_of_coldest_month REAL;
ALTER TABLE YEARLY_WEATHER ADD COLUMN temp_annual_range REAL;
ALTER TABLE YEARLY_WEATHER ADD COLUMN precip_wettest_month;
ALTER TABLE YEARLY_WEATHER ADD COLUMN precip_driest_month;
ALTER TABLE YEARLY_WEATHER ADD COLUMN isothermality;
ALTER TABLE YEARLY_WEATHER ADD COLUMN mean_temperature_of_wettest_quarter REAL;
ALTER TABLE YEARLY_WEATHER ADD COLUMN mean_temperature_of_driest_quarter REAL;
ALTER TABLE YEARLY_WEATHER ADD COLUMN mean_temperature_of_warmest_quarter REAL;
ALTER TABLE YEARLY_WEATHER ADD COLUMN mean_temperature_of_coldest_quarter REAL;
ALTER TABLE YEARLY_WEATHER ADD COLUMN precip_of_wettest_quarter REAL;
ALTER TABLE YEARLY_WEATHER ADD COLUMN precip_of_driest_quarter REAL;
ALTER TABLE YEARLY_WEATHER ADD COLUMN precip_of_warmest_quarter REAL;
ALTER TABLE YEARLY_WEATHER ADD COLUMN precip_of_coldest_quarter REAL;

ALTER TABLE YEARLY_WEATHER ADD COLUMN  precip_seasonality REAL;
ALTER TABLE YEARLY_WEATHER ADD COLUMN  temp_seasonality REAL;

UPDATE YEARLY_WEATHER
SET 
  max_temp_of_warmest_month = (
    SELECT MAX(m.month_avg_max_temp)
    FROM MONTHLY_WEATHER_VARS AS m
    WHERE m.year = YEARLY_WEATHER.year
  ),
  min_temp_of_coldest_month = (
    SELECT MIN(m.month_avg_min_temp)
    FROM MONTHLY_WEATHER_VARS AS m
    WHERE m.year = YEARLY_WEATHER.year
  ),
  precip_wettest_month = (
    SELECT MAX(m.month_total_precip)
    FROM MONTHLY_WEATHER_VARS AS m
    WHERE m.year = YEARLY_WEATHER.year
  ),
  precip_driest_month = (
    SELECT MIN(m.month_total_precip)
    FROM MONTHLY_WEATHER_VARS AS m
    WHERE m.year = YEARLY_WEATHER.year
  );
UPDATE YEARLY_WEATHER
SET 
  temp_annual_range = ROUND(max_temp_of_warmest_month - min_temp_of_coldest_month,2);

UPDATE YEARLY_WEATHER
SET 
  isothermality = (
    SELECT ROUND((m.mean_diurnal_range/temp_annual_range)*100,2)
    FROM MONTHLY_WEATHER_VARS AS m
    WHERE m.year = YEARLY_WEATHER.year
  );

UPDATE YEARLY_WEATHER
SET mean_temperature_of_wettest_quarter = (
    SELECT q.quarter_avg_mean_temp
    FROM QUARTERLY AS q
    WHERE q.year = YEARLY_WEATHER.year
    ORDER BY q.quarter_total_precip DESC
    LIMIT 1),
    
    mean_temperature_of_driest_quarter = (
    SELECT q.quarter_avg_mean_temp
    FROM QUARTERLY AS q
    WHERE q.year = YEARLY_WEATHER.year
    ORDER BY q.quarter_total_precip
    LIMIT 1),

    mean_temperature_of_warmest_quarter = (
    SELECT MAX(q.quarter_avg_mean_temp)
    FROM QUARTERLY AS q
    WHERE q.year = YEARLY_WEATHER.year),

    mean_temperature_of_coldest_quarter = (
    SELECT MIN(q.quarter_avg_mean_temp)
    FROM QUARTERLY AS q
    WHERE q.year = YEARLY_WEATHER.year),

    precip_of_wettest_quarter = (
    SELECT MAX(q.quarter_total_precip)
    FROM QUARTERLY AS q
    WHERE q.year = YEARLY_WEATHER.year),
    
    precip_of_driest_quarter = (
    SELECT MIN(q.quarter_total_precip)
    FROM QUARTERLY AS q
    WHERE q.year = YEARLY_WEATHER.year),

    precip_of_warmest_quarter = (
    SELECT q.quarter_total_precip
    FROM QUARTERLY AS q
    WHERE q.year = YEARLY_WEATHER.year
    ORDER BY q.quarter_avg_mean_temp DESC
    LIMIT 1),

    precip_of_coldest_quarter = (
    SELECT q.quarter_total_precip
    FROM QUARTERLY AS q
    WHERE q.year = YEARLY_WEATHER.year
    ORDER BY q.quarter_avg_mean_temp
    LIMIT 1);

    UPDATE YEARLY_WEATHER
    SET precip_seasonality=(
    SELECT 
      ROUND(
      SQRT(
          SUM(POWER(m.month_total_precip - yearly.avg_precip,2))/(12-1))/yearly.avg_precip, 4)
    FROM MONTHLY_WEATHER_VARS AS m
    JOIN (
      SELECT year, AVG(month_total_precip) AS avg_precip
      FROM MONTHLY_WEATHER_VARS
      GROUP BY year
    ) AS yearly
    ON m.year = yearly.year
    WHERE m.year = YEARLY_WEATHER.year);

    UPDATE YEARLY_WEATHER AS y
    SET temp_seasonality = (
        SELECT ROUND(
                   SQRT(
                       SUM(POWER(m.month_avg_mean_temp - y.year_avg_mean_temp, 2)) / (12 - 1)
                   ) * 100, 4)
        FROM MONTHLY_WEATHER_VARS AS m
        WHERE m.year = y.year
    );
SELECT * FROM YEARLY_WEATHER ORDER BY year LIMIT 10;

 * sqlite:///WEATHER.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
155 rows affected.
155 rows affected.
155 rows affected.
155 rows affected.
155 rows affected.
155 rows affected.
Done.


year,group_size,year_avg_max_temp,year_avg_min_temp,year_avg_mean_temp,year_total_precip,max_temp_of_warmest_month,min_temp_of_coldest_month,temp_annual_range,precip_wettest_month,precip_driest_month,isothermality,mean_temperature_of_wettest_quarter,mean_temperature_of_driest_quarter,mean_temperature_of_warmest_quarter,mean_temperature_of_coldest_quarter,precip_of_wettest_quarter,precip_of_driest_quarter,precip_of_warmest_quarter,precip_of_coldest_quarter,precip_seasonality,temp_seasonality
1871,184,14.55,6.06,10.32,489.5,26.51,-11.04,37.55,126.3,33.0,24.69,19.83,0.81,19.83,0.81,254.8,234.7,254.8,234.7,0.2523,812.3523
1872,366,11.95,2.44,7.2,1049.5,27.45,-14.11,41.56,162.8,36.1,20.02,0.01,14.19,21.06,-6.53,367.0,140.4,332.7,209.4,0.4601,1257.329
1873,365,10.84,1.68,6.27,1046.45,26.48,-13.45,39.93,168.5,35.9,18.31,-0.06,12.54,18.91,-6.52,329.7,188.7,220.95,307.1,0.4602,1163.9836
1874,365,9.59,0.71,5.16,980.1,25.07,-14.72,39.79,137.45,42.0,16.56,-7.34,-0.36,18.54,-7.34,278.9,166.65,266.15,278.9,0.3844,1169.7037
1875,365,7.69,-1.54,3.08,999.74,24.64,-18.59,43.23,153.46,27.8,18.2,16.58,-11.42,16.58,-11.42,351.14,169.0,351.14,169.0,0.4394,1273.3514
1876,366,8.86,-0.25,4.31,952.08,25.52,-15.85,41.37,114.92,42.8,19.39,-8.0,-2.54,17.04,-8.0,264.1,201.01,262.48,264.1,0.2971,1185.8915
1877,365,9.75,0.49,5.13,885.68,25.13,-18.2,43.33,117.58,36.62,21.42,0.3,10.5,17.4,-7.9,251.31,189.6,205.18,239.59,0.3565,1089.3223
1878,365,10.04,1.12,5.59,1049.48,25.13,-16.48,41.61,125.59,28.24,22.71,0.34,-7.09,17.68,-7.09,354.02,187.7,268.64,187.7,0.3696,1103.125
1879,365,8.35,-1.48,3.44,1004.1,23.32,-16.93,40.25,115.5,43.48,22.34,15.43,9.23,15.43,-9.97,288.13,207.9,288.13,241.28,0.3151,1163.8911
1880,366,9.25,-0.96,4.15,915.06,23.92,-14.57,38.49,135.34,44.24,31.15,-2.09,-7.91,16.36,-7.91,268.87,202.51,228.56,202.51,0.3282,1106.5404


## Add Degree Days to daily averages table

In [349]:
%%sql
ALTER TABLE DAY_AVERAGE_WEATHER ADD COLUMN degree_days REAL;

 * sqlite:///WEATHER.db
Done.


[]

In [350]:
%%sql

UPDATE DAY_AVERAGE_WEATHER
SET degree_days = ROUND(((avg_max_temp + avg_min_temp)/2)-8, 2);

SELECT * FROM DAY_AVERAGE_WEATHER LIMIT 10

 * sqlite:///WEATHER.db
56166 rows affected.
Done.


date,group_size,avg_max_temp,avg_min_temp,avg_mean_temp,avg_total_precip,degree_days
1871-07-01,1,28.3,13.3,20.8,0.0,12.8
1871-07-02,1,26.1,19.4,22.8,0.0,14.75
1871-07-03,1,28.9,11.7,20.3,0.0,12.3
1871-07-04,1,26.7,17.2,22.0,0.0,13.95
1871-07-05,1,27.8,16.7,22.3,19.6,14.25
1871-07-06,1,28.3,16.7,22.5,0.0,14.5
1871-07-07,1,23.3,20.6,22.0,19.1,13.95
1871-07-08,1,30.0,16.7,23.4,0.0,15.35
1871-07-09,1,26.7,21.1,23.9,1.0,15.9
1871-07-10,1,24.4,15.0,19.7,0.0,11.7


## Save Average Dataset to CSV

In [352]:
query1 = "SELECT * FROM DAY_AVERAGE_WEATHER"
data_frame1 = pd.read_sql(query1, con)

csv_file_name1 = 'QC_daily_weather_data.csv'
data_frame1.to_csv(csv_file_name1, index=False)
print(f"Data exported to '{csv_file_name1}' successfully.")

query2 = "SELECT * FROM YEARLY_WEATHER"
data_frame2 = pd.read_sql(query2, con)

csv_file_name2 = 'QC_yearly_weather_data.csv'
data_frame2.to_csv(csv_file_name2, index=False)
print(f"Data exported to '{csv_file_name2}' successfully.")

Data exported to 'QC_daily_weather_data.csv' successfully.
Data exported to 'QC_yearly_weather_data.csv' successfully.
